In [1]:
from lib.utilities import *
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
############### User Configuration ###############
##################################################

datadir = "." # All ZDA files in this directory + subdirectories will be loaded
selected_filename = "05_01_05" # focus for subsequent analysis

In [3]:
############## Driver script: begin ##############
##################################################

# Load data
data_loader = DataLoader()
data_loader.load_all_zda(data_dir=datadir + "/zda_targets")

# Select data of interest        
selected_data = data_loader.select_data_by_keyword(selected_filename)

# Clip to the time range 20 - 90 ms (get rid of the camera "foot", look at prestim only)
if selected_data is not None:
    
    selected_data.clip_data(t_range=[40,180])
    raw_data, meta, rli = selected_data.get_data(trial=0), selected_data.get_meta(), selected_data.get_rli()

    # Need to subtract off the low-frequency voltage drift. First-order correction
    tr = Tracer()
    tr.correct_background(meta, raw_data)

    #full trace
    tr.plot_trace(raw_data[:,:,:], 
               40, 
               40, 
               meta['interval_between_samples'])
else:
    print("Filename containing '" + selected_filename + "' not found.")

./zda_targets/05_01_05.zda
Number of files loaded: 1


In [4]:
# Frequency Decompositions

freq_analzyer = FreqAnalyzer()

x_fft = freq_analzyer.compute_fft_binning(meta)
y_fft_avg = None
n = raw_data.shape[0] * raw_data.shape[1]

for x in range(raw_data.shape[0]):
    for y in range(raw_data.shape[1]):
            
            y_fft = freq_analzyer.decompose_trace_frequencies(meta, 
                                                raw_data[x,y,:],
                                                x_fft=x_fft,
                                                lower_freq=0,
                                                upper_freq=50,
                                                y_max=2000,
                                                plot=False)
            if y_fft_avg is None:
                y_fft_avg = y_fft / n
            else:
                y_fft_avg += y_fft / n

In [5]:
# Mean frequency decomposition (averaged by frequency over all pixels)
plt.plot(x_fft, 
         np.abs(y_fft_avg))
plt.xlim([10, 400])
plt.ylim([500,600])
plt.title("Average Frequency Decomposition: All Pixels")
plt.show()
plt.savefig(selected_filename + "_avg_freq.jpg")

In [6]:
# Average only for high-SNR and/or ROIs

trial = selected_data.get_data(trial=0)
asnr = AnalyzerSNR(trial)

snr = asnr.get_snr(plot=True)

print("max SNR:", np.max(snr), 
      "\nmin SNR:", np.min(snr))

max SNR: 82.9422746304119 
min SNR: 29.022573069329045


In [7]:
# cluster by SNR and display
asnr.cluster_on_snr(plot=True)

array([[4, 3, 3, ..., 1, 1, 3],
       [3, 3, 3, ..., 1, 3, 1],
       [3, 3, 3, ..., 1, 1, 1],
       ...,
       [2, 4, 1, ..., 1, 4, 4],
       [4, 1, 1, ..., 1, 4, 1],
       [4, 4, 4, ..., 4, 4, 1]])

In [10]:
 # k=0 is highest SNR cluster (indices returned)
highest_snr_cluster = asnr.get_kth_cluster(0, plot=True)
avg_snr_by_cluster = asnr.get_average_snr_by_cluster()
print(avg_snr_by_cluster, highest_snr_cluster)

[52.15538164389097, 60.02277505201125, 46.82505055769709, 57.851747766868996] [[ 8 12 15 ... 78 78 78]
 [45 43 52 ... 62 64 65]]


In [14]:
# Frequency Decompositions for 3 highest-SNR clusters
for k in range(3):
    snr_cluster = asnr.get_kth_cluster(k, plot=False)
    freq_analzyer = FreqAnalyzer()

    x_fft = freq_analzyer.compute_fft_binning(meta)
    y_fft_avg = None
    n = raw_data.shape[0] * raw_data.shape[1]

    for i in range(snr_cluster.shape[1]):
        x = snr_cluster[0][i]
        y = snr_cluster[1][i]

        y_fft = freq_analzyer.decompose_trace_frequencies(meta, 
                                            raw_data[x,y,:],
                                            x_fft=x_fft,
                                            lower_freq=0,
                                            upper_freq=50,
                                            y_max=2000,
                                            plot=False)
        if y_fft_avg is None:
            y_fft_avg = y_fft / n
        else:
            y_fft_avg += y_fft / n

    # Mean frequency decomposition (averaged by frequency over all pixels)
    plt.plot(x_fft, 
             np.abs(y_fft_avg),
             label=str(k+1) +"th-highest SNR cluster")
plt.xlim([10, 400])
plt.ylim([100,250])
plt.legend()
plt.title("Average Frequency Decomposition: SNR Clusters")
plt.show()
plt.savefig(selected_filename + "_all_freq.jpg")